## 1. About the Data

This dataset contains information about international student numbers , their enrolments, nationality, field of study etc.

Student numbers data is available from year 2002 till October 2020. However the data related to enrolments, commencements and field of study is available for 2019 and 2020 (till October).


The data is available in 2 csv files mentioned below:

1. studentsPublic.csv     ---> This file provides the actual student numbers in Australia by nationality and State.
2. nationalitySummary.csv ---> This file provides details about field of study, total enrolments and commencements.

In [41]:
## Read the csv files .
import pandas as pd
df1 = pd.read_csv('../../data/studentPublic.csv')
df2 = pd.read_csv('../../data/nationalitySummary.csv')

In [42]:
## Lets examine the columns for both the files.

all_cols = [list(df1.columns), list(df2.columns)]
all_cols

[['Year', 'Month', 'Nationality', 'State', 'Students', 'Share', 'Growth'],
 ['Year',
  'Month',
  'Sector',
  'Nationality',
  'Level Of Study',
  'Broad Field Of Education',
  'Enrolments',
  'Commencements',
  'EnrolmentsShare',
  'EnrolmentsGrowth',
  'CommencementsShare',
  'CommencementsGrowth']]

In [43]:
## Get an inersection to get the columns common in both the dataframes.

my_function = lambda x, y: set(x).intersection(set(y))
my_function(df1.columns, df2.columns)

{'Month', 'Nationality', 'Year'}

### It is clear from the above output that both the datasets provide completely different attributes related to international students in Australia. The columns common to both the dataseta are Month, Nationality and Year only. So, it is more appropriate to clean and analyse the datasets separately.

## 2. Inspecting and Cleaning the Student numbers data (studentsPublic.csv)

In [44]:
## Import the required modules

import pandas as pd
import numpy as np
from functools import reduce
pd.set_option('display.max_columns', None)

In [45]:
## Read the csv file and load into a dataframe.

df_students = pd.read_csv('../../data/studentPublic.csv')

In [46]:
## check the type

type(df_students)

pandas.core.frame.DataFrame

In [47]:
## Check the number of rows and columns in the student dataset.

df_students.shape

(23819, 7)

In [48]:
## Get some information about the dataframe created earlier.

df_students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23819 entries, 0 to 23818
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         23819 non-null  int64  
 1   Month        23819 non-null  object 
 2   Nationality  23819 non-null  object 
 3   State        23819 non-null  object 
 4   Students     23819 non-null  int64  
 5   Share        23819 non-null  int64  
 6   Growth       21358 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.3+ MB


In [49]:
## Check the list of columns present in the dataset.

df_students.columns

Index(['Year', 'Month', 'Nationality', 'State', 'Students', 'Share', 'Growth'], dtype='object')

In [50]:
# Normalising  the columns.
df_students.columns = [col.lower() for col in df_students.columns]
df_students.columns

Index(['year', 'month', 'nationality', 'state', 'students', 'share', 'growth'], dtype='object')

In [51]:
## Get the first few rows at the beginning of the data set.

df_students.head(10)

,year,month,nationality,state,students,share,growth
0,2020,Oct,_All,_All,677369,100,-8.0
1,2020,Oct,_All,NSW,257386,38,-8.0
2,2020,Oct,_All,NT,3925,1,25.0
3,2020,Oct,_All,SA,36751,5,1.0
4,2020,Oct,_All,TAS,13290,2,-2.0
5,2020,Oct,_All,ACT,15437,2,-6.0
6,2020,Oct,_All,QLD,98847,15,-13.0
7,2020,Oct,_All,VIC,222398,33,-8.0
8,2020,Oct,_All,WA,38206,6,-9.0
9,2020,Oct,Argentina,_All,1395,0,13.0


In [52]:
## Get few rows at the end of the data set.

df_students.tail(10)

,year,month,nationality,state,students,share,growth
23809,2009,Dec,Sweden,NSW,669,0,-3.0
23810,2009,Dec,Lithuania,NSW,54,0,-10.0
23811,2010,Dec,Benin,NSW,4,0,0.0
23812,2009,Dec,Ghana,NSW,20,0,0.0
23813,2010,Dec,Libya,NSW,279,0,258.0
23814,2010,Dec,Montenegro,NSW,4,0,-80.0
23815,2010,Dec,Canada,NSW,1314,1,-3.0
23816,2010,Dec,Colombia,NSW,2022,1,-9.0
23817,2010,Dec,Tajikistan,NSW,4,0,50.0
23818,2010,Dec,Cameroon,NSW,4,0,0.0


In [53]:
## Get the count of missing values for all the coloumns

df_students.isna().sum()

year              0
month             0
nationality       0
state             0
students          0
share             0
growth         2461
dtype: int64

We can see that all the columns have no missing values except last column, 'Growth'. This was mostly expected as the dataset is pre-processed available on the Department of Education website. 

In [54]:
## The possible reason for missing 'growth'  column could be attributed to the fact that there may be no students for the previous year.
## lets check this out

null_data = df_students[df_students.isnull().any(axis=1)]
null_data

,year,month,nationality,state,students,share,growth
33,2020,Oct,Antigua and Barbuda,_All,4,0,NaN
86,2020,Oct,Chad,_All,4,0,NaN
88,2020,Oct,French Polynesia,_All,4,0,NaN
100,2020,Oct,New Caledonia,_All,4,0,NaN
141,2020,Oct,Marshall Islands,_All,4,0,NaN
...,...,...,...,...,...,...,...
23772,2010,Dec,Afghanistan,TAS,4,0,NaN
23776,2008,Dec,Haiti,VIC,4,0,NaN
23785,2009,Dec,Morocco,SA,4,0,NaN
23790,2010,Dec,Uruguay,TAS,4,0,NaN


In [55]:
## Get the data for one country (say Chad) where growth column is null/missing.

null_data = null_data[(null_data.nationality == 'Chad') & (null_data.state == '_All')]
null_data.sort_values(by = 'year')

,year,month,nationality,state,students,share,growth
6010,2007,Dec,Chad,_All,4,0,NaN
4379,2015,Dec,Chad,_All,4,0,NaN
86,2020,Oct,Chad,_All,4,0,NaN


## Dealing with missing values
It can be seen from the above outout that growth column is null for an year where there is no data for the previous year.
Hence it is reasonable to fill the missing values with 0 for the growth column

In [56]:
## Filling the missing values with 0

df_students = df_students.fillna(0)

In [57]:
## Get the count of missing values for all the coloumns after 'fillna' operation

df_students.isna().sum()

year           0
month          0
nationality    0
state          0
students       0
share          0
growth         0
dtype: int64

### Great, there are no missing values in the student dataset. Let us export the clean data into a new csv (studentsPublic_Clean.csv)

In [58]:
len(df_students)

23819

In [59]:
df_students.to_csv('../../data/studentPublic_Clean.csv', index=False)

In [60]:
## Read the cleaned csv file and load into a new dataframe.

df_students_clean = pd.read_csv('../../data/studentPublic_Clean.csv')

In [61]:
dups = df_students_clean.duplicated()
dups.sum()

0

In [19]:
df_students_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23819 entries, 0 to 23818
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         23819 non-null  int64  
 1   Month        23819 non-null  object 
 2   Nationality  23819 non-null  object 
 3   State        23819 non-null  object 
 4   Students     23819 non-null  int64  
 5   Share        23819 non-null  int64  
 6   Growth       23819 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.3+ MB


## 3. Inspecting and Cleaning the Student numbers data (nationalitySummary.csv)

In [62]:
## Read the csv file and load into a dataframe.

df_enrol = pd.read_csv('../../data/nationalitySummary.csv')

In [21]:
## check the type

type(df_enrol)

pandas.core.frame.DataFrame

In [63]:
## Check the number of rows and columns in the student dataset.

df_enrol.shape

(13079, 12)

In [22]:
## Get some information about the dataframe created earlier.

df_enrol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year                      13079 non-null  int64  
 1   Month                     13079 non-null  object 
 2   Sector                    13079 non-null  object 
 3   Nationality               13079 non-null  object 
 4   Level Of Study            13079 non-null  object 
 5   Broad Field Of Education  13079 non-null  object 
 6   Enrolments                13079 non-null  int64  
 7   Commencements             13079 non-null  int64  
 8   EnrolmentsShare           13079 non-null  int64  
 9   EnrolmentsGrowth          12091 non-null  float64
 10  CommencementsShare        12755 non-null  float64
 11  CommencementsGrowth       11050 non-null  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 1.2+ MB


In [64]:
## Check the list of columns present in the dataset.

df_enrol.columns

Index(['Year', 'Month', 'Sector', 'Nationality', 'Level Of Study',
       'Broad Field Of Education', 'Enrolments', 'Commencements',
       'EnrolmentsShare', 'EnrolmentsGrowth', 'CommencementsShare',
       'CommencementsGrowth'],
      dtype='object')

In [65]:
# Normalising  the columns.
df_enrol.columns = [col.lower() for col in df_enrol.columns]
df_enrol.columns

Index(['year', 'month', 'sector', 'nationality', 'level of study',
       'broad field of education', 'enrolments', 'commencements',
       'enrolmentsshare', 'enrolmentsgrowth', 'commencementsshare',
       'commencementsgrowth'],
      dtype='object')

In [66]:
## Get the first few rows at the beginning of the data set.

df_enrol.head(10)

,year,month,sector,nationality,level of study,broad field of education,enrolments,commencements,enrolmentsshare,enrolmentsgrowth,commencementsshare,commencementsgrowth
0,2019,Dec,_All,_All,_All,_All,956773,515082,100,0.0,100.0,0.0
1,2019,Dec,_All,Canada,_All,_All,4717,2328,0,1.0,0.0,-5.0
2,2019,Dec,_All,Gaza Strip and West Bank,_All,_All,30,10,0,-21.0,0.0,-23.0
3,2019,Dec,_All,Dominican Republic,_All,_All,7,4,0,-50.0,0.0,-80.0
4,2019,Dec,_All,Oman,_All,_All,1192,494,0,-1.0,0.0,-10.0
5,2019,Dec,_All,Costa Rica,_All,_All,70,30,0,-16.0,0.0,-29.0
6,2019,Dec,_All,St Kitts and Nevis,_All,_All,4,4,0,0.0,0.0,-100.0
7,2019,Dec,_All,Zambia,_All,_All,340,142,0,-11.0,0.0,-13.0
8,2019,Dec,_All,Uruguay,_All,_All,165,109,0,40.0,0.0,38.0
9,2019,Dec,_All,Grenada,_All,_All,4,4,0,-50.0,0.0,-100.0


In [31]:
## Get few rows at the end of the data set.

df_enrol.tail(10)

,year,month,sector,nationality,level of study,broad field of education,enrolments,commencements,enrolmentsshare,enrolmentsgrowth,commencementsshare,commencementsgrowth
13069,2020,Oct,VET,Zimbabwe,_All,Engineering and Related Technologies,25,13,9,25.0,10.0,8.0
13070,2020,Oct,VET,Zimbabwe,_All,Creative Arts,4,4,0,-67.0,1.0,-50.0
13071,2020,Oct,VET,Zimbabwe,_All,"Agriculture, Environmental and Related Studies",4,4,1,-43.0,0.0,-100.0
13072,2020,Oct,VET,Zimbabwe,_All,Education,6,4,2,0.0,2.0,-25.0
13073,2020,Oct,VET,Zimbabwe,_All,Information Technology,10,4,4,-23.0,3.0,-64.0
13074,2020,Oct,VET,Zimbabwe,_All,Architecture and Building,10,4,4,-38.0,2.0,-63.0
13075,2020,Oct,VET,Zimbabwe,_All,Health,43,14,16,-48.0,11.0,-48.0
13076,2020,Oct,VET,Zimbabwe,_All,"Food, Hospitality and Personal Services",11,6,4,10.0,5.0,0.0
13077,2020,Oct,VET,Zimbabwe,_All,Management and Commerce,78,40,29,-2.0,32.0,-5.0
13078,2020,Oct,VET,Zimbabwe,_All,Society and Culture,80,40,30,-18.0,32.0,-17.0


In [67]:
## Get the count of missing values for all the coloumns

df_enrol.isna().sum()

year                           0
month                          0
sector                         0
nationality                    0
level of study                 0
broad field of education       0
enrolments                     0
commencements                  0
enrolmentsshare                0
enrolmentsgrowth             988
commencementsshare           324
commencementsgrowth         2029
dtype: int64

In [68]:
## lets check this out for missing values.

null_data1 = df_enrol[df_enrol.isnull().any(axis=1)]
null_data1

,year,month,sector,nationality,level of study,broad field of education,enrolments,commencements,enrolmentsshare,enrolmentsgrowth,commencementsshare,commencementsgrowth
22,2019,Dec,_All,Niger,_All,_All,4,4,0,100.0,0.0,NaN
43,2019,Dec,_All,Eritrea,_All,_All,5,4,0,-17.0,0.0,NaN
73,2019,Dec,_All,Benin,_All,_All,4,4,0,0.0,0.0,NaN
81,2019,Dec,_All,French Guiana,_All,_All,4,4,0,0.0,0.0,NaN
89,2019,Dec,_All,Zaire,_All,_All,4,4,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13034,2020,Oct,Higher Education,Zambia,_All,Education,4,4,2,0.0,2.0,NaN
13037,2020,Oct,Higher Education,Zambia,_All,Architecture and Building,4,4,1,-33.0,0.0,NaN
13041,2020,Oct,Higher Education,Zambia,_All,Mixed Field Programmes,4,4,1,NaN,3.0,NaN
13048,2020,Oct,VET,Zambia,_All,Architecture and Building,4,4,2,0.0,4.0,NaN


In [69]:
## Get the data for one country (say Niger) where enrolmentsgrowth column is null/missing.

null_data1 = null_data1[(null_data1.nationality == 'Niger')]
null_data1.sort_values(by = 'year')

,year,month,sector,nationality,level of study,broad field of education,enrolments,commencements,enrolmentsshare,enrolmentsgrowth,commencementsshare,commencementsgrowth
22,2019,Dec,_All,Niger,_All,_All,4,4,0,100.0,0.0,NaN
696,2019,Dec,Higher Education,Niger,_All,_All,4,4,100,100.0,100.0,NaN
2422,2019,Dec,Higher Education,Niger,Bachelor Degree,_All,4,4,50,0.0,0.0,NaN
2423,2019,Dec,Higher Education,Niger,Masters Degree (Coursework),_All,4,4,50,NaN,100.0,NaN
8051,2019,Dec,Higher Education,Niger,_All,Natural and Physical Sciences,4,4,50,NaN,100.0,NaN
8052,2019,Dec,Higher Education,Niger,_All,Society and Culture,4,4,50,0.0,0.0,NaN
2731,2020,Oct,_All,Niger,_All,_All,4,4,0,100.0,0.0,NaN
2944,2020,Oct,ELICOS,Niger,_All,_All,4,4,100,NaN,100.0,NaN
5074,2020,Oct,ELICOS,Niger,Non AQF Award,_All,4,4,100,NaN,100.0,NaN
11833,2020,Oct,ELICOS,Niger,_All,Society and Culture,4,4,100,NaN,100.0,NaN


## Dealing with missing values

It can be seen from the above outout that enrolmentsgrowth column is null for an year where there is no data for the previous year. e.g. 'Non AQF Award' has no enrolments in 2019. So, this is null for 2020.
Hence it is reasonable to fill the missing values with 0.

In [70]:
## Filling the missing values with 0

df_enrol = df_enrol.fillna(0)

In [71]:
## Get the count of missing values for all the coloumns after 'fillna' operation

df_enrol.isna().sum()

year                        0
month                       0
sector                      0
nationality                 0
level of study              0
broad field of education    0
enrolments                  0
commencements               0
enrolmentsshare             0
enrolmentsgrowth            0
commencementsshare          0
commencementsgrowth         0
dtype: int64

## Great, there are no missing values in the enrolments dataset. Let us export the clean data into a new csv (nationalitySummary_Clean.csv)

In [29]:
len(df_enrol)

13079

In [72]:
df_enrol.to_csv('../../data/nationalitySummary_Clean.csv', index=False)

In [73]:
## Read the cleaned csv file and load into a new dataframe.

df_enrol_clean = pd.read_csv('../../data/nationalitySummary_Clean.csv')

In [74]:
dups = df_enrol_clean.duplicated()
dups.sum()

0

In [75]:
df_enrol_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079 entries, 0 to 13078
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      13079 non-null  int64  
 1   month                     13079 non-null  object 
 2   sector                    13079 non-null  object 
 3   nationality               13079 non-null  object 
 4   level of study            13079 non-null  object 
 5   broad field of education  13079 non-null  object 
 6   enrolments                13079 non-null  int64  
 7   commencements             13079 non-null  int64  
 8   enrolmentsshare           13079 non-null  int64  
 9   enrolmentsgrowth          13079 non-null  float64
 10  commencementsshare        13079 non-null  float64
 11  commencementsgrowth       13079 non-null  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 1.2+ MB


In [76]:
df_enrol_clean.isna().sum()

year                        0
month                       0
sector                      0
nationality                 0
level of study              0
broad field of education    0
enrolments                  0
commencements               0
enrolmentsshare             0
enrolmentsgrowth            0
commencementsshare          0
commencementsgrowth         0
dtype: int64